In [1]:
import pandas as pd
import json
import csv
import io
import requests
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import json
from datetime import datetime
import time
import numpy as np
from itertools import islice

In [2]:
from shapely.geometry import Polygon

In [3]:
import matplotlib.pyplot as plt

In [4]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

### 20220607 making 5 and 25 minute transit isochrones

In [21]:
allboxes = pd.read_csv('20220520_jan_21_jan_21_may_22_unified_edited.csv')
makechrone = allboxes[(allboxes.in_2022 == True) | (allboxes.in_nov_2020 == True)]

In [22]:
makechrone = makechrone.reset_index()

In [105]:
def get_isochrone(index,lat,long):
    # three different time ticks
    df = gpd.GeoDataFrame()
    for travel_time in [300, 1500]:
        headers = {
            'Accept': 'application/geo+json',
            'X-Application-Id': '',
            'X-Api-Key': '',
        }

        json_data = {
            'arrival_searches': [
                {
                    'id': str(index),
                    'coords': {
                        'lat': lat, 
                        'lng': long,
                    },
                    'transportation': {
                        'type': 'public_transport',
                    },
                    'arrival_time': '2022-05-05T18:01:00.000Z',
                    'travel_time': travel_time,
                },
            ],
        }

        response = requests.post('https://api.traveltimeapp.com/v4/time-map', headers=headers, json=json_data)
        geojson = json.dumps(response.json())
        polygon = gpd.read_file(geojson)
        polygon['travel_time'] = travel_time
        polygon['lat'] = lat
        polygon['long'] = long
        df = df.append(polygon)
    return (df)

In [24]:
input1 = list(np.arange(0,len(makechrone)))

In [25]:
newlist = []

In [26]:
for i in range(0,len(input1),10):
    x=i
    newlist.append(input1[x:x+10])

In [106]:
masterdf = gpd.GeoDataFrame()

In [107]:
for listi in newlist:
    for i in listi:
        try:
            index = i
            lat = makechrone['lat'].iloc[i]
            long = makechrone['long'].iloc[i]
            result = get_isochrone(index, lat, long)
#             time.sleep(1)
            masterdf = masterdf.append(result)
            print (i)
        except Exception:
            print('error')
            print (i)
    time.sleep(60)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [29]:
boundary = gpd.read_file('tl_2020_us_county/tl_2020_us_county.shp')
GA = boundary[boundary.STATEFP == '13'].copy()
GA = GA.to_crs('EPSG:4326')

In [108]:
masterdf.head()

,geometry,travel_time,lat,long
0,"MULTIPOLYGON (((-85.11684 34.91381, -85.11466 ...",900,34.915487,-85.106602
0,"MULTIPOLYGON (((-84.14317 34.22048, -84.14425 ...",900,34.223824,-84.139148
0,"MULTIPOLYGON (((-83.30033 30.86759, -83.29929 ...",900,30.869999,-83.292707
0,"MULTIPOLYGON (((-83.99109 34.52928, -83.98891 ...",900,34.535023,-83.981607
0,"MULTIPOLYGON (((-84.35082 31.32362, -84.35135 ...",900,31.317829,-84.342261


In [111]:
masterdf1 =masterdf.copy()
masterdf1['dropbox_point'] = gpd.points_from_xy(masterdf1.long, masterdf1.lat, crs="EPSG:4326")
masterdf1pts = masterdf1.rename(columns = {'geometry':'isochrone','dropbox_point':'geometry'})
masterdf1pts.crs = 'EPSG:4326'
# find out which county each drop box belongs to
masterdf2 = gpd.sjoin(masterdf1pts, GA, how = 'left', op = 'within')

In [112]:
masterdf2[pd.isna(masterdf2.COUNTYFP) == True]

,isochrone,travel_time,lat,long,geometry,index_right,STATEFP,COUNTYFP,COUNTYNS,GEOID,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON


In [113]:
masterdf2 = masterdf2.drop(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'NAME', 'NAMELSAD', 'LSAD',
       'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'], axis = 1)
masterdf2 = masterdf2.rename(columns = {'geometry':'drop_box_loc'})

In [114]:
GA1 = GA.rename(columns = {'geometry':'county_boundary'}).iloc[:,[3,17]].copy()
GA1 = GA1.reset_index()

In [115]:
# GA1 = GA.rename(columns = {'geometry':'county_boundary'})[['GEOID','county_boundary']].copy()
masterdf3 = pd.merge(masterdf2, GA1, on = 'GEOID', how = 'outer')
masterdf3 = masterdf3.rename(columns = {'isochrone':'geometry'})
masterdf3 = masterdf3.reset_index()

In [117]:
clipped_isochrones = gpd.GeoDataFrame()

In [118]:
for i in list(np.arange(0,len(masterdf3))):
    try:
        gdf = gpd.GeoDataFrame(masterdf3.iloc[[i]])
        gdf.crs ='EPSG:4326'
        clipping = gpd.GeoDataFrame([1], geometry=[(masterdf3.county_boundary.iloc[i])], crs='EPSG:4326') 
        clipped = gpd.clip(gdf,clipping)
        clipped_isochrones = clipped_isochrones.append(clipped)
        print (i)
    except Exception:
        print('error')
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [124]:
clipped_isochrones.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 361 entries, 0 to 360
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   level_0          361 non-null    int64   
 1   geometry         361 non-null    geometry
 2   travel_time      361 non-null    int64   
 3   lat              361 non-null    float64 
 4   long             361 non-null    float64 
 5   index_right      361 non-null    int64   
 6   GEOID            361 non-null    object  
 7   index            361 non-null    int64   
 8   county_boundary  361 non-null    geometry
dtypes: float64(2), geometry(2), int64(4), object(1)
memory usage: 28.2+ KB


In [146]:
allboxesyr = allboxes[['lat','long','in_2022','in_2021','in_nov_2020']].copy()

In [147]:
allboxesyr.lat = np.round(allboxesyr.lat,6)
allboxesyr.long = np.round(allboxesyr.long,6)

In [148]:
clipped_isochrones.lat = np.round(clipped_isochrones.lat,6)
clipped_isochrones.long = np.round(clipped_isochrones.long,6)

In [149]:
clipped_isochrones_1 = clipped_isochrones.merge(allboxesyr, on = ['lat','long'], how = 'outer')

In [172]:
clipped_isochrones_2 = clipped_isochrones_1[~((pd.isna(clipped_isochrones_1.in_2022) == True) & (pd.isna(clipped_isochrones_1.in_nov_2020) == True))]

In [153]:
len(clipped_isochrones_2)

361

In [155]:
old_file = gpd.read_file('20220519_transit_isochrones_clipped_by_county_full_info.shp')

In [157]:
old_file.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   lat         1194 non-null   float64 
 1   long        1194 non-null   float64 
 2   travel_tim  1194 non-null   float64 
 3   index_righ  1194 non-null   float64 
 4   GEOID       1194 non-null   object  
 5   NAMELSAD    1194 non-null   object  
 6   in_2022     564 non-null    float64 
 7   in_2021     762 non-null    float64 
 8   in_nov_202  888 non-null    float64 
 9   geometry    1194 non-null   geometry
dtypes: float64(7), geometry(1), object(2)
memory usage: 93.4+ KB


In [173]:
clipped_isochrones_2 =clipped_isochrones_2.drop(['level_0','index','index_right'], axis = 1)

In [174]:
clipped_isochrones_2 = clipped_isochrones_2.rename(columns = {'travel_time':'travel_tim'})

In [175]:
clipped_isochrones_2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 361 entries, 0 to 360
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   geometry         361 non-null    geometry
 1   travel_tim       361 non-null    float64 
 2   lat              361 non-null    float64 
 3   long             361 non-null    float64 
 4   GEOID            361 non-null    object  
 5   county_boundary  361 non-null    geometry
 6   in_2022          187 non-null    object  
 7   in_2021          221 non-null    object  
 8   in_nov_2020      295 non-null    object  
dtypes: float64(3), geometry(2), object(4)
memory usage: 28.2+ KB


In [176]:
new_file = pd.concat([old_file,clipped_isochrones_2])

In [179]:
new_file.drop(['index_righ','county_boundary'], axis = 1).to_file('20220521_transit_isochrones_10_15_20_30.shp')

C:\Users\jhuo\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [ ]:
# there was a problem with joining the 900 isochrones with years present to need to redo

In [5]:
new_file = gpd.read_file('.shp')

In [7]:
new_file.travel_tim.value_counts()

1800.0    399
1200.0    398
600.0     397
900.0     361
Name: travel_tim, dtype: int64

In [10]:
newfile1 = new_file.drop(['in_2022','in_2021','in_nov_202','in_nov_2_1'], axis = 1).copy()

In [11]:
newfile1.lat = np.round(newfile1.lat,6)
newfile1.long = np.round(newfile1.long,6)

In [12]:
allboxes = pd.read_csv('20220520_jan_21_jan_21_may_22_unified_edited.csv')

In [13]:
allboxes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             393 non-null    int64  
 1   address_may_22         186 non-null    object 
 2   lat                    393 non-null    float64
 3   long                   393 non-null    float64
 4   in_2022                187 non-null    object 
 5   address_jan_21         250 non-null    object 
 6   in_2021                253 non-null    object 
 7   nov_2020_form_address  295 non-null    object 
 8   in_nov_2020            295 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 27.8+ KB


In [14]:
allboxesyrs = allboxes[['lat','long','in_2022','in_2021','in_nov_2020']].copy().drop_duplicates()

In [15]:
allboxesyrs.lat = np.round(allboxesyrs.lat,6)
allboxesyrs.long = np.round(allboxesyrs.long,6)

In [17]:
newfile2 = newfile1.merge(allboxesyrs, on = ['lat','long'], how = 'left')

In [29]:
newfile2 = newfile2.drop_duplicates()

In [34]:
timecnt = pd.DataFrame(newfile2.groupby(['lat','long']).travel_tim.count())

In [35]:
timecnt[timecnt.travel_tim != 4]

,,travel_tim
lat,long,
30.878741,-84.208053,5
31.157196,-81.386360,3
31.207757,-81.507695,3
31.232103,-84.209399,6
31.292901,-82.842967,3
31.541500,-82.464370,3
31.604322,-85.048853,3
31.847487,-81.596222,3
32.066152,-82.896421,3


In [48]:
newfile2[(newfile2.lat == 34.001103)]
# this was likely the duluth address that was edited out

,lat,long,travel_tim,GEOID,NAMELSAD,geometry,in_2022,in_2021,in_nov_2020
114,34.001103,-84.161852,600.0,13135,Gwinnett County,"POLYGON ((-84.16855 34.00308, -84.16909 34.003...",NaN,NaN,NaN
115,34.001103,-84.161852,1200.0,13135,Gwinnett County,"POLYGON ((-84.17244 34.00445, -84.17136 34.005...",NaN,NaN,NaN
116,34.001103,-84.161852,1800.0,13135,Gwinnett County,"POLYGON ((-84.17244 34.00445, -84.17136 34.005...",NaN,NaN,NaN


In [54]:
newfile2[(newfile2.lat == 31.232103)]
# this was likely the duluth address that was edited out

,lat,long,travel_tim,GEOID,NAMELSAD,geometry,in_2022,in_2021,in_nov_2020
967,31.232103,-84.209399,600.0,13205,Mitchell County,"POLYGON ((-84.21613 31.23022, -84.21666 31.230...",True,True,True
968,31.232103,-84.209399,1200.0,13205,Mitchell County,"POLYGON ((-84.21798 31.23069, -84.21798 31.231...",True,True,True
969,31.232103,-84.209399,1800.0,13205,Mitchell County,"POLYGON ((-84.21798 31.23069, -84.21798 31.231...",True,True,True
970,31.232103,-84.209399,1200.0,13205,Mitchell County,"POLYGON ((-84.21798 31.23069, -84.21798 31.231...",True,True,True
971,31.232103,-84.209399,1800.0,13205,Mitchell County,"POLYGON ((-84.21798 31.23069, -84.21798 31.231...",True,True,True
1492,31.232103,-84.209399,900.0,13205,None,"POLYGON ((-84.21798 31.23069, -84.21798 31.231...",True,True,True


In [67]:
newfile2.to_file('.shp')

C:\Users\jhuo\AppData\Local\Continuum\anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
